In [14]:
from math import log
import numpy as np
from time import time
from gensim import corpora, models

In [15]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Вычисляем перплексию модели:

In [16]:
def perplexity(model, corpus):
    corpus_length = 0
    log_likelihood = 0
    topic_profiles = model.state.get_lambda() / np.sum(model.state.get_lambda(), axis=1)[:, np.newaxis]
    for document in corpus:
        gamma, _ = model.inference([document])
        document_profile = gamma / np.sum(gamma)
        for term_id, term_count in document:
            corpus_length += term_count
            term_probability = np.dot(document_profile, topic_profiles[:, term_id])
            log_likelihood += term_count * log(term_probability)
    perplexity = np.exp(-log_likelihood / corpus_length)
    return perplexity

Составляем коллекцию текстов из документа без учёта стоп-слов:

In [17]:
with open("US_proc_l_b.txt") as f:
    text = f.read()
    us_texts = text.split("\n")
    stop = ['и', 'а', 'когда', 'но', 'с', 'к', 'чтобы', 'же', 'ли', 'во', 'над', 'ну', 'два',
            'вот', 'более', 'потому', 'быть', 'конечно', 'перед', 'да', 'какая', 'после', 'где', 'сам', 'еще', 'ещё', 
            'куда', 'даже', 'можно', 'раз', 'или', 'чего', 'об', 'зачем', 'по', 'все', 'мой', 'моя', 'моё', 
            'мои', 'одна', 'одно', 'без', 'как', 
            'чтоб', 'в','нет', 'разве', 'впрочем', 'на', 'такой', 'будто', 'при', 'ни', 'когда', 'за', 
            'со', 'у', 'уж', 'три', 'о', 'про', 'из','чем', 'для', 'до', 'под', 'с', 'чуть', 'если', 'ведь',
            'один', 'хоть', 'бы', 'уже', 'не', 'быть', 'есть', 'через', 'только', 'от', 'между', 'чтобы', 'ж',
            'тоже', '-', 'б', 'ибо','сильвио', 'кольбер', 'джой','граф', 'графиня', 'пчёлы',  'р', 'анна', 
            'николаевна', 'я', 'ты', 'он', 'она', 'оно', 'мы', 'вы', 'они', 'его', 'ее', 'её', 'ему', 'ей', 
            'меня', 'ваше', 'сиятельство', 'маша', 'их', 'вам', 'него', 'мне', 'мною', 'ней', 
            'нее', 'тебя', 'меня', 'ним', 'нас', 'нему', 'нем', 'кольбера', 'мое', 'моё', 'кузька', 
            'вера', 'мою', 'африканович', 'ермолай', 'нюшка', 'степановна', 'колюша', 'арина', 'иван',
            'нам', 'вас', 'г', 'им', 'парменко', 'катерина', 'евстолья', 'сосновка',  'г', 'им', 'кочерга',
            'пчела', 'наш', 'ваш', 'пять', 'шесть']
    stops = set(stop)
    for i in range(len(us_texts)):
        us_texts[i]=us_texts[i].split()
        us_texts[i] = [w for w in us_texts[i] if not w in stops]
    print(len(us_texts))

123


Составляем словарь по коллекции и фильтруем его:

In [19]:
dictionary = corpora.Dictionary(us_texts)
print ('Original: {}'.format(dictionary))
dictionary.filter_extremes(no_below = 2, no_above = 0.5, keep_n=None)
dictionary.save('ap3.dict')
print ('Filtered: {}'.format(dictionary))

Original: Dictionary(3619 unique tokens: ['особый', 'зелень', 'сени', 'стая', 'седло']...)
Filtered: Dictionary(1183 unique tokens: ['особый', 'приходиться', 'оба', 'свинцовый', 'вода']...)


In [20]:
corpus = [dictionary.doc2bow(text) for text in us_texts]
corpora.MmCorpus.serialize('ap3.mm', corpus) # store on disc

Обучаем модель, выводим её перплексию:

In [21]:
start = time()
model = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=15, chunksize=15, update_every=1, passes=2)
print ('Evaluation time: {}'.format((time()-start) / 60))
print ('Perplexity: {}'.format(perplexity(model, corpus)))

Evaluation time: 0.004189014434814453
Perplexity: 462.10309341059104


Смотрим на получившиеся темы:

In [22]:
for topic in range(0, 15):
    words = [i[0] for i in model.show_topic(topic, topn = 10)]
    print("topic", topic+1, "top 10: ", ", ".join(words))

topic 1 top 10:  взять, точно, зверок, который, обыкновенно, крыльцо, лошадь, этот, сорок, глаз
topic 2 top 10:  домой, очевидно, хотеть, спрашивать, кто, там, это, себя, списывать, танцевать
topic 3 top 10:  это, сквозь, то, оставаться, нельзя, город, тот, дело, так, год
topic 4 top 10:  гора, мочь, лететь, забор, служба, какой, отдаление, баба, пойти, час
topic 5 top 10:  свой, кирпич, жена, ворота, <question-mark>, это, столб, открывать, казаться, вокруг
topic 6 top 10:  ребенок, который, любить, редко, должный, ко, вместе, отдавать, свой, просьба
topic 7 top 10:  окно, знать, выламывать, дом, последний, стекло, колонна, фасад, что, девочка
topic 8 top 10:  <question-mark>, что, <exclamation-mark>, говорить, теперь, <ellipsis>, свой, сказать, тут, седой
topic 9 top 10:  время, это, чувствовать, становиться, замечать, вода, очень, этот, девка, однако
topic 10 top 10:  потом, мерин, девка, забывать, надевать, пармен, стыдно, обед, сперва, запирать
topic 11 top 10:  ночь, час, приходит

Записываем в csv top-10 слов каждой темы:

In [23]:
top = open('US-LDAtopics.csv', 'w', encoding='utf-8')
n=1
for topic in range(0, 15):
    for i in model.show_topic(topic, topn = 10):
        #line = n,',', i[0], ',', i[1], '\n'
        line = str(n), str(i[0]), str(i[1])
        line = ','.join(line)+'\n'
        top.write(line)
    n+=1
top.close()

Также записываем в отдельный файл слова, которые относятся к каждой теме с вероятностью не меньше 0.005:

In [24]:
clear = open ('UST_clear.txt', 'w', encoding='utf-8')
for topic in range(0, 15):
    for i in model.show_topic(topic, topn = 10):
        if i[1]>0.005:
            line = str(i[0])+' '
            clear.write(line)
    clear.write('\n')
clear.close()